<a href="https://colab.research.google.com/github/fadingautumn/DissertationProject/blob/working_with_json/New_accentizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 6.4 MB/s 


In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import nltk
import pymorphy2
import re
import difflib
import collections
from collections import Counter
from nltk import word_tokenize
from utils import Phonetic
phonetic = Phonetic('words_accent.json.bz2')
morph = pymorphy2.MorphAnalyzer()

dictmeter = {
    '010101': 'Трёхстопный ямб',
    '01010101': 'Четырёхстопный ямб',
    '0101010101': 'Пятистопный ямб',
    '010101010101': 'Шестистопный ямб',
    '01010101010101': 'Семистопный ямб',
    '0101010101010101': 'Восьмистопный ямб',
    '101010': 'Трёхстопный хорей',
    '10101010': 'Четырёхстопный хорей',
    '1010101010': 'Пятистопный хорей',
    '101010101010': 'Шестистопный хорей',
    '10101010101010': 'Семистопный хорей',
    '1010101010101010': 'Восьмистопный хорей',
    '100100100': 'Трёхстопный дактиль',
    '100100100100': 'Четырёхстопный дактиль',
    '100100100100100': 'Пятистопный дактиль',
    '100100100100100100': 'Шестистопный дактиль',
    '010010010': 'Трёхстопный амфибрахий',
    '010010010010': 'Четырёхстопный амфибрахий',
    '010010010010010': 'Пятистопный амфибрахий',
    '010010010010010010': 'Шестистопный амфибрахий',
    '001001001': 'Трёхстопный анапест',
    '001001001001': 'Четырёхстопный анапест',
    '001001001001001': 'Пятистопный анапест',
    '001001001001001001': 'Шестистопный анапест'
    
}

text = open('oneg.txt', encoding = 'utf-8').read()

In [4]:
def get_numbers(token):
  syl_count = phonetic.syllables_count(token)
  accent_syl = phonetic.accent_syllable(token)
  substr = ''
  for i in range(syl_count):
    i = '0'
    substr += i
  position = accent_syl - 1
  acc_symb = '1'
  substr = substr[:position] + acc_symb + substr[position+1:]
  return substr


In [7]:
def get_verse_size_number(rawpoem):
    lines = rawpoem.split('\n')
    supersize = []
    for line in lines:
        cleantext = line.lower() 
        cleantext = re.sub('[^а-я]', ' ', cleantext)
        words = nltk.word_tokenize(cleantext)
        razmer = ''
        for word in words:
            razmer += get_numbers(word)
        if razmer != '' and len(razmer) < 19:
          supersize.append(razmer)
    return supersize

In [10]:
def best_verse_size(num_lines):
  line_size_list = []
  for i in get_verse_size_number(num_lines):
    temp = 0
    best_fit_size = ''
    for key in dictmeter:
      output = int(difflib.SequenceMatcher(None, i, key).ratio()*100)
      if temp < output and temp < 100:
        temp = output
        best_fit_size = dictmeter[key]
    line_size_list.append(best_fit_size)
  counter = Counter(line_size_list)
  max_count = counter.most_common(2)
  max_count1 = f'{max_count[0][0]} встречается в {max_count[0][1]} строках стихотворения, а значит, он наиболее вероятен. '
  max_count2 = f'Также программа выделила {max_count[1][0]}, который встречается в {max_count[1][1]} строках стихотворения.'
  return max_count1 + max_count2

best_verse_size(text)

'Трёхстопный дактиль встречается в 1074 строках стихотворения, а значит, он наиболее вероятен. Также программа выделила Четырёхстопный ямб, который встречается в 999 строках стихотворения.'